# Methylation Annotation Comparison
- Compare results using Pyensembl vs Illumina Methylation 450 probe data

In [1]:
import glob
import pandas as pd
import biomart as bm
import json
import pyensembl
import sys
import subprocess
import os

In [2]:
def shell_do(command, log=False, return_log=False, make_part=False):
    print(f'Executing: {(" ").join(command.split())}', file=sys.stderr)

    if make_part == False:
        res = subprocess.run(command.split(), stdout=subprocess.PIPE)
    else:
        res = subprocess.run(command, shell=True, stdout=subprocess.PIPE)

    if log:
        print(res.stdout.decode('utf-8'))
    if return_log:
        return (res.stdout.decode('utf-8'))

## 1. mQTL read in
mQTL msmr files are already annotated using Illumina probe data

In [3]:
# glob for SMR results and filter for mMeta + mcrae
msmr_files = glob.glob("/../omicSynth/intermediate_results/*.msmr")

# filter for files we care about
mmeta = [x for x in msmr_files if 'mMeta' in x]

mcrae = [x for x in msmr_files if 'mcrae' in x]

# merge all mQTL paths into one list
mqtl_paths = mmeta + mcrae

# remove updated files
mqtl_paths = [x for x in mqtl_paths if 'update' not in x]

In [4]:
# read in paths and create central df
mqtl_df = pd.DataFrame()

for path in mqtl_paths:
    tmp = pd.read_csv(path, sep = '\t')
    
    # add columns with path data
    tmp['path'] = path
    
    # concat main df and tmp df
    mqtl_df = pd.concat([mqtl_df,tmp])

In [5]:
mqtl_df

,probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,...,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_SMR_multi,p_HEIDI,nsnp_HEIDI,path
0,cg05505459,1,C1orf170,911995,rs3737728,1,1021415,A,G,0.264826,...,0.657000,0.114014,8.291478e-09,0.159665,0.095581,0.094825,0.094825,NaN,NaN,/data/CARD_AA/projects/omicSynth/v8/intermedia...
1,cg15899727,1,NaN,1003529,rs3737728,1,1021415,A,G,0.264826,...,0.392314,0.069718,1.831792e-08,0.267388,0.160394,0.095500,0.095500,NaN,NaN,/data/CARD_AA/projects/omicSynth/v8/intermedia...
2,cg07181843,1,NaN,1009445,rs3737728,1,1021415,A,G,0.264826,...,-0.486032,0.068586,1.375945e-12,-0.215830,0.127350,0.090118,0.090118,NaN,NaN,/data/CARD_AA/projects/omicSynth/v8/intermedia...
3,cg01912388,1,NaN,1011560,rs3737728,1,1021415,A,G,0.264826,...,0.500445,0.068389,2.523275e-13,0.209613,0.123462,0.089546,0.089546,NaN,NaN,/data/CARD_AA/projects/omicSynth/v8/intermedia...
4,cg14854474,1,NaN,1012526,rs3737728,1,1021415,A,G,0.264826,...,0.510518,0.068247,7.407435e-14,0.205478,0.120886,0.089175,0.089175,NaN,NaN,/data/CARD_AA/projects/omicSynth/v8/intermedia...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4611,cg05270750,16,PRDM7,90143788,rs2241038,16,90088904,G,C,0.431493,...,0.349461,0.033298,9.122853e-26,0.018056,0.025269,0.474875,0.863688,0.266772,20.0,/data/CARD_AA/projects/omicSynth/v8/intermedia...
4612,cg04298253,16,PRDM7,90143815,rs2241038,16,90088904,G,C,0.431493,...,0.308830,0.033423,2.463572e-20,0.020432,0.028613,0.475172,0.657352,0.270835,20.0,/data/CARD_AA/projects/omicSynth/v8/intermedia...
4613,cg16611967,16,NaN,90144006,rs4427799,16,90087927,T,A,0.432515,...,0.348006,0.033212,1.086492e-25,0.017758,0.025200,0.481005,0.795437,0.248801,20.0,/data/CARD_AA/projects/omicSynth/v8/intermedia...
4614,cg09435890,16,NaN,90144788,rs112579477,16,90080747,A,C,0.051125,...,-0.956548,0.058338,2.022491e-60,-0.001976,0.018954,0.916975,0.589344,0.745538,20.0,/data/CARD_AA/projects/omicSynth/v8/intermedia...


In [11]:
len(mqtl_df[mqtl_df.Gene.isna() == True].probeID.unique())

41391

~ There are 974523 rows with no gene name and 41391 unique probes with no gene names ~

## 2. Pyensembl
- use TSS for gene annotation = genes_at_locus(contig, position, end=None, strand=None)
- ran python as swarm, just reading in

In [6]:
mqtl_anno = pd.read_csv('/../omicSynth/v8/analysis/mQTL_anno.csv')

In [7]:
paths = list(mqtl_df['path'])
len(paths)

3298773

In [8]:
# combine original df for paths
mqtl_anno['path'] = paths

In [9]:
# fix gene names that are a list in pyensembl genes
tmp = []

for x in mqtl_anno['Gene_pyensembl']:
    if ',' in x:
        res = x.split(',')
        if len(set(res)) > 1:
            tmp.append(res[0])
        elif len(set(res)) == 1:
            tmp.append(res[0])
    else:
        tmp.append(x)

# replace column with fixed values
mqtl_anno['Gene_pyensembl'] = tmp

In [10]:
mqtl_anno.Gene = mqtl_anno.Gene.fillna('none')

In [11]:
# fix gene names that are a list in previous genes
tmp = []
for x in mqtl_anno['Gene']:
    if ';' in x:
        res = x.split(';')
        if len(set(res)) > 1:
            tmp_set = ','.join(map(str,list(set(res))))
            tmp.append(tmp_set)
        elif len(set(res)) == 1:
            tmp.append(res[0])
    else:
        tmp.append(x)

# replace column with fixed values
mqtl_anno['Gene'] = tmp

In [12]:
# pull only results that were able to get a gene name
good = mqtl_anno[mqtl_anno.Gene_pyensembl != 'novel_or_none']

# create mapping
py_map = dict(zip(good['probeID'], good['Gene_pyensembl']))

# number of rows before mapping
mqtl_anno[mqtl_anno.Gene.isna() == True].shape

(0, 24)

In [13]:
# create mapping for probes that already had names
good_orig = mqtl_anno[mqtl_anno.Gene != 'none']

orig_map = dict(zip(good_orig['probeID'], good_orig['Gene']))

In [14]:
# see if there are any common probes between the two dictionaries
list(set(orig_map.keys()).intersection(set(good.keys())))

[]

In [15]:
# combine the two dictionaries into one
master_map = {**orig_map , **py_map}

# create tmp column to store mapped gene values
mqtl_anno['Gene_update'] = mqtl_anno['probeID'].map(master_map)

# fill in any values that we couldn't find gene names for
mqtl_anno['Gene_update'] = mqtl_anno['Gene_update'].fillna('novel_or_none')

In [16]:
# number of rows with NaN values after mapping
mqtl_anno[mqtl_anno.Gene.isna() == True].shape

(0, 25)

In [ ]:
print(mqtl_anno.Gene_pyensembl.value_counts())
print(mqtl_anno.Gene_update.value_counts())

In [17]:
# drop old gene columns
mqtl_anno.drop(['Gene', 'Gene_pyensembl'], axis = 1, inplace = True)
# rename
mqtl_anno.rename({'Gene_update': 'Gene'}, axis = 1, inplace = True)
# reposition Gene column
mqtl_anno = mqtl_anno.iloc[:,[0,1,-1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]]

In [18]:
mqtl_anno

,probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,...,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_SMR_multi,p_HEIDI,nsnp_HEIDI,path
0,cg05505459,1,C1orf170,911995,rs3737728,1,1021415,A,G,0.264826,...,0.657000,0.114014,8.291478e-09,0.159665,0.095581,0.094825,0.094825,NaN,NaN,/data/CARD_AA/projects/omicSynth/v8/intermedia...
1,cg15899727,1,RP11-465B22.3,1003529,rs3737728,1,1021415,A,G,0.264826,...,0.392314,0.069718,1.831792e-08,0.267388,0.160394,0.095500,0.095500,NaN,NaN,/data/CARD_AA/projects/omicSynth/v8/intermedia...
2,cg07181843,1,RNF223,1009445,rs3737728,1,1021415,A,G,0.264826,...,-0.486032,0.068586,1.375945e-12,-0.215830,0.127350,0.090118,0.090118,NaN,NaN,/data/CARD_AA/projects/omicSynth/v8/intermedia...
3,cg01912388,1,novel_or_none,1011560,rs3737728,1,1021415,A,G,0.264826,...,0.500445,0.068389,2.523275e-13,0.209613,0.123462,0.089546,0.089546,NaN,NaN,/data/CARD_AA/projects/omicSynth/v8/intermedia...
4,cg14854474,1,novel_or_none,1012526,rs3737728,1,1021415,A,G,0.264826,...,0.510518,0.068247,7.407435e-14,0.205478,0.120886,0.089175,0.089175,NaN,NaN,/data/CARD_AA/projects/omicSynth/v8/intermedia...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3298768,cg05270750,16,PRDM7,90143788,rs2241038,16,90088904,G,C,0.431493,...,0.349461,0.033298,9.122853e-26,0.018056,0.025269,0.474875,0.863688,0.266772,20.0,/data/CARD_AA/projects/omicSynth/v8/intermedia...
3298769,cg04298253,16,PRDM7,90143815,rs2241038,16,90088904,G,C,0.431493,...,0.308830,0.033423,2.463572e-20,0.020432,0.028613,0.475172,0.657352,0.270835,20.0,/data/CARD_AA/projects/omicSynth/v8/intermedia...
3298770,cg16611967,16,PRDM7,90144006,rs4427799,16,90087927,T,A,0.432515,...,0.348006,0.033212,1.086492e-25,0.017758,0.025200,0.481005,0.795437,0.248801,20.0,/data/CARD_AA/projects/omicSynth/v8/intermedia...
3298771,cg09435890,16,PRDM7,90144788,rs112579477,16,90080747,A,C,0.051125,...,-0.956548,0.058338,2.022491e-60,-0.001976,0.018954,0.916975,0.589344,0.745538,20.0,/data/CARD_AA/projects/omicSynth/v8/intermedia...


In [20]:
print(mqtl_anno.Gene.value_counts())

novel_or_none      676754
PTPRN2              13618
MAD1L1               7696
ADARB2               5880
PRDM16               5445
                    ...  
RPPH1                   2
IGBP1P1                 2
SNAR-B2,SNAR-B1         2
CTGLF11P                2
PTPRC                   2
Name: Gene, Length: 20204, dtype: int64


In [22]:
# export out csv files 
final_paths = list(mqtl_anno.path.unique())

for path in final_paths:
    # remove msmr file extension
    name = path.split('.msmr')[0]
    
    # query main df
    tmp = mqtl_anno.query(f'path == "{path}"')
    
    # drop paths column
    tmp.drop('path', axis = 1, inplace = True)
    
    # export
    tmp.to_csv(f'{path}_update.csv', sep = '\t', index = None)
    